# Local environment checks

***Before running this notebook, you might want to use Jupyter's "clear output" function to erase the results of the previous execution of this notebook. That will make more apparent what has been executed in the current session.***

This notebook serves as a tool for verifying the proper configuration and functionality of your local Marlowe Runtime installation. If you are utilizing the [demeter.run](https://demeter.run/) development platform, you can skip this.

📌 **Note**: You can watch a video that guides you through this Jupyter notebook [here (TODO)](https://youtu.be/TODO).

Should you have any inquiries about Marlowe, feel free to ask questions in the [IOG Discord's #ask-marlowe channel](https://discord.com/channels/826816523368005654/936295815926927390), or if you encounter any challenges with this lesson, you can report them in the [Marlowe Starter Kit GitHub repository's issues section](https://github.com/input-output-hk/marlowe-starter-kit/issues).


## Command Line Tools and Environment

This section introduces the `check-tools-and-env.sh` script, located in the *scripts* folder. This script is designed to validate that the required command line tools and environment settings are correctly set.

You can delve into the *scripts* folder to find different helpers that are used accross the different notebooks.

In [ ]:
export SCRIPTS=../scripts
source $SCRIPTS/check-tools-and-env.sh

## Check the Health of the Marlowe Runtime Services

### Cardano Node

When `cardano-node` is started, it may take minutes or hours to synchronize with the tip of the blockchain, depending upon the network. Wait until querying the node shows that its `syncProgress` is 100%.


In [ ]:
cardano-cli query tip --testnet-magic "$CARDANO_TESTNET_MAGIC" | json2yaml

###  Marlowe Runtime Indexers

Check to see that `marlowe-chain-indexer` has reached the tip of the blockchain.

In [ ]:
psql chain_preprod -c 'select max(slotno) from chain.block;'

Also check to see that `marlowe-indexer` has *approximately* reached the tip of the blockchain.

In [ ]:
psql chain_preprod -c 'select max(slotno) from marlowe.block;'

*Troubleshooting note:* If the `marlowe-chain-indexer` or `marlowe-indexer` query result is `null` or an extremely low number such as `-1`, you may need to reset the docker postgres volume. You can do that from the HOST machine:

```bash
docker-compose stop postgres
docker-compose remove postgres
docker volume rm marlowe-starter-kit-postgres
docker-compose up -d
```


### Marlowe Runtime Proxy Service

Check that the `marlowe-runtime-cli` command can communicate with the Marlowe Runtime backend services by querying the history of one of the Marlowe contracts that has previously been executed on the blockchain.

In [ ]:
if [ "${CARDANO_TESTNET_MAGIC}" == "1" ]; then
  export "CONTRACT_ID=2e515845d04e18bda77d48010948ca891870baf1a2e68d3262d22ec0af4f6f5e#1"
elif [ "${CARDANO_TESTNET_MAGIC}" == "2" ]; then
  export "CONTRACT_ID=da3c4c1d6f0c052fb149e9bc0600f0869e0fbc1d7c1ae455c0d2cb1ed9717c30#1"
else
  export "CONTRACT_ID=a1e0b1ff9743fc501f5e07f8db1ef9b148ea9e31f1ea190b1afaebc59ec0faac#1"
fi

marlowe-runtime-cli log $CONTRACT_ID

We can also see the contract in the [Marlowe Scan tool](https://docs.marlowe.iohk.io/docs/getting-started/marlowe-scan)

In [ ]:
$SCRIPTS/marlowe-scan.sh $CONTRACT_ID

### Marlowe Runtime Web Server

Check that one can communicate with the Marlowe Runtime web server and receive a `200 OK` response.


In [ ]:
curl -sSI "$MARLOWE_RT_WEBSERVER_URL/healthcheck"